In [1]:
import pandas as pd
import numpy as np
import jieba
#定义停用词、标点符号
punctuation = ["，","。","","；", "？"]
#定义语料
jdcomment = pd.read_excel('D:/泰迪杯/c题/附件1/酒店评论.xlsx')
jdscore = pd.read_excel('D:/泰迪杯/c题/附件2/酒店评分.xlsx')

In [2]:
jdcomment.index=jdcomment['酒店名称']

In [3]:
z1 = [f'H{x}{y}' for x in range(0,5) for y in range(0,10)][1:]
df1 = [jdcomment.loc[f'{x}']['评论内容'] for x in z1]

In [4]:
df2 = [jdcomment.loc['H50']['评论内容']]

In [5]:
x = df1+df2
x=[str(i) for i in x]

In [6]:
import re
#加载停用词
stoplist= [word.strip() for word in open('stopwords.txt',encoding='gb2312').readlines()]
# print(stoplist)
sentences_list = x
# 对句子进行分词
def seg_depart(sentence):
    # 去掉非汉字字符
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = jieba.cut(sentence.strip())
    word_list = []
    for word in sentence_depart:
        if word not in stoplist:
            word_list.append(word)
    # 如果句子整个被过滤掉了，如：'02-2717:56'被过滤，那就返回[],保持句子的数量不变
    return word_list

sentence_word_list = []
for sentence in sentences_list:
    line_seg = seg_depart(sentence)
    sentence_word_list.append(line_seg)

x = sentences_list

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELL\AppData\Local\Temp\jieba.cache
Loading model cost 0.759 seconds.
Prefix dict has been built successfully.


In [7]:
x[0]

'酒店名称\nH01                                            酒店很适合家庭出行\nH01                                        升级了房间 延迟退房 很赞\nH01    这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...\nH01                                               酒店很好不错\nH01                                                超五星好评\n                             ...                        \nH01                                            游泳池很好服务不错\nH01                                            豪华的装修服务不错\nH01                                             房间不错前台很帅\nH01                                              态度不错的酒店\nH01                                               服务不错不错\nName: 评论内容, Length: 1129, dtype: object'

In [8]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
 
#该类会统计每个词语的tf-idf权值
tf_idf_transformer = TfidfVectorizer()

x1 = tf_idf_transformer.fit_transform(x)
type(x1)

scipy.sparse.csr.csr_matrix

In [9]:
x1.shape

(50, 1392)

In [10]:
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV

In [15]:
dt = DecisionTreeClassifier(max_depth = None)
bc = BaggingClassifier(dt, n_estimators = 20, max_samples = 1.0, max_features = 1.0)

In [16]:
clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),
                                     ),
                   param_grid)
clf.fit(x_train,y_train.astype('str'))

D:\Anacnda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
D:\Anacnda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
D:\Anacnda\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                               min_samples_split=2,
                                              

In [13]:
#服务得分评分
y = jdscore
y1 = y['服务得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y1,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
model_BaggingRegressor.fit(x_train,y_train)
y_hat = model_BaggingRegressor.predict(x_test)
print("服务得分评分：",y_hat.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat,y_test))

服务得分评分： 4.619999999999999
代价函数MSE： 0.016279999999999874


In [15]:
#位置得分评分
y = jdscore
y2 = y['位置得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y2,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
model_BaggingRegressor.fit(x_train,y_train)
y_hat1 = model_BaggingRegressor.predict(x_test)
print("位置评分：",y_hat1.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat1,y_test))

位置评分： 4.7299999999999995
代价函数MSE： 0.004370000000000005


In [16]:
#设施得分评分
y = jdscore
y3 = y['设施得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y3,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
model_BaggingRegressor.fit(x_train,y_train)
y_hat2 = model_BaggingRegressor.predict(x_test)
print("设施评分：",y_hat2.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat2,y_test))

设施评分： 4.640000000000001
代价函数MSE： 0.03708000000000007


In [17]:
#设施得分评分
y = jdscore
y4 = y['卫生得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y4,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
model_BaggingRegressor.fit(x_train,y_train)
y_hat3 = model_BaggingRegressor.predict(x_test)
print("卫生评分：",y_hat3.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat3,y_test))

卫生评分： 4.77
代价函数MSE： 0.006639999999999975


In [18]:
#设性价比得分评分
y = jdscore
y5 = y['性价比得分']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y5,test_size=0.2)
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
model_BaggingRegressor.fit(x_train,y_train)
y_hat4 = model_BaggingRegressor.predict(x_test)
print("性价比评分：",y_hat4.min())
#均方误差 MSE
from sklearn.metrics import mean_squared_error as MSE
print("代价函数MSE：",MSE(y_hat4,y_test))

性价比评分： 4.0
代价函数MSE： 0.006280000000000024
